In [ ]:
#  #########################################################################
#  #                           IN THE NAME OF ALLAH                        #
#  #      STEEL REBAR DIAMETER AND CONCRETE CONFINED SECTION COLUMN        #
#  #            OPTIMIZATION ANALYSIS WITH NONLINEAR DYNAMIC               #
#  #                         NEWTON-RAPHSON METHOD                         #
#  #-----------------------------------------------------------------------#
#  #              THIS PROGRAM WRITTEN BY SALAR DELAVAR QASHQAI            #
#  #                   EMAIL: salar.d.ghashghaei@gmail.com                 #
#  #########################################################################

In [37]:
#import the os module
import os
import math
import time

In [ ]:
#to create a directory at specified path with name "Data"
os.mkdir('C:\\OPENSEESPY_SALAR')
#this will create the directory with name 'Data' and will update it when we rerun the analysis,
# otherwise we have to keep deleting the old 'Data' Folder
dir = "C:\\OPENSEESPY_SALAR\\OPENSEESPY_DATA"
if not os.path.exists(dir):
    os.makedirs(dir)

In [38]:
# Your existing code
def MAXABS_FUN(DATA_FILE):
    import numpy as np
    # Read and process displacement data
    NameFiles = DATA_FILE
    filename = f"{NameFiles}.txt"
    D = np.loadtxt(filename)
    #print(D)
    MAXABS = np.max(np.abs([D[:, 1]]))
    #print("MAX. ABS. :", MAXABS)
    return MAXABS

In [ ]:
# pip install openseespy

In [ ]:
# OUTPUT DATA ADDRESS:
SALAR_DIR = 'C:/OPENSEESPY_SALAR/OPENSEESPY_DATA/';

In [39]:
#
#    ^Y
#    |
#    2       __ 
#    |          | 
#    |          |
#    |          |
#  (1)       LCol
#    |          |
#    |          |
#    |          |
#  =1=      _|_  -------->X
#

# SET UP ----------------------------------------------------------------------------


#########################################################################################################################################################################

def DYNAMIC_ANALYSIS(HCol, BCol, LCol, coverCol, Weight, numBarsCol, BD, fc):
    import openseespy.opensees as op
    op.wipe()
    op.model('basic', '-ndm', 2, '-ndf', 3) 
    PCol =Weight  # nodal dead-load weight per column
    g =  9810 # mm/s^2
    Mass =  PCol/g

    #ACol = BCol*HCol;  # cross-sectional area, make stiff
    IzCol = (BCol* HCol**3)/12; # Column moment of inertia
    # nodal coordinates:
    op.node(1, 0.0, 0.0) # node#, X, Y
    op.node(2, 0.0, LCol)
    # Single point constraints -- Boundary Conditions
    op.fix(1, 1, 1, 1) # node DX DY RZ
    # node#, Mx My Mz, Mass=Weight/g, neglect rotational inertia at nodes
    op.mass(2, Mass, 1e-9, 0.0)
    
    barAreaCol = (3.1415 * BD**2) / 4  # area of longitudinal-reinforcement bars
    
    ColSecTag = 1			# assign a tag number to the column section
    # MATERIAL parameters -------------------------------------------------------------------
    IDconcCore = 1; 				# material ID tag -- confined core concrete
    IDconcCover = 2; 				# material ID tag -- unconfined cover concrete
    IDreinf = 3; 				# material ID tag -- reinforcement
    
    # nominal concrete compressive strength
    Ec = 4700 * math.sqrt(-fc) # Concrete Elastic Modulus (the term in sqr root needs to be in psi

    # confined concrete
    Kfc = 1.3;			# ratio of confined to unconfined concrete strength
    fc1C = Kfc*fc;		# CONFINED concrete (mander model), maximum stress
    eps1C = 2*fc1C/Ec;	# strain at maximum stress 
    fc2C = 0.2*fc1C;		# ultimate stress
    eps2C = 5*eps1C;		# strain at ultimate stress 
    # unconfined concrete
    fc1U = fc;			# UNCONFINED concrete (todeschini parabolic model), maximum stress
    eps1U = -0.0025;			# strain at maximum strength of unconfined concrete
    fc2U = 0.2*fc1U;		# ultimate stress
    eps2U = -0.012;			# strain at ultimate stress
    Lambda = 0.1;				# ratio between unloading slope at $eps2 and initial slope $Ec
    # tensile-strength properties
    ftC = -0.55*fc1C;		# tensile strength +tension
    ftU = -0.55*fc1U;		# tensile strength +tension
    Ets = ftU/0.002;		# tension softening stiffness

    Fy = 4000			# STEEL yield stress
    Cy = 0.02			# STEEL yield stress
    Es = Fy/Cy				# modulus of steel
    Bs = 0.01				# strain-hardening ratio 
    R0 = 18.0				# control the transition from elastic to plastic branches
    cR1 = 0.925				# control the transition from elastic to plastic branches
    cR2 = 0.15				# control the transition from elastic to plastic branches

    op.uniaxialMaterial('Concrete02', IDconcCore, fc1C, eps1C, fc2C, eps2C, Lambda, ftC, Ets) # build cover concrete (confined)
    op.uniaxialMaterial('Concrete02', IDconcCover, fc1U, eps1U, fc2U, eps2U, Lambda, ftU, Ets) # build cover concrete (unconfined)
    op.uniaxialMaterial('Steel02', IDreinf, Fy, Es, Bs, R0,cR1,cR2) # build reinforcement material
    # FIBER SECTION properties -------------------------------------------------------------
    # symmetric section
    #                        y
    #                        ^
    #                        |     
    #             ---------------------     --   --
    #             |   o  o   o   o    |     |    -- cover
    #             |                   |     |
    #             |   o           o   |     |
    #    z <---   |          +        |     H
    #             |   o           o   |     |
    #             |                   |     |
    #             |   o  o    o   o   |     |    -- cover
    #             ---------------------     --   --
    #             |-------- B --------|
    #
    # RC section: 

    coverY = HCol/2.0	# The distance from the section z-axis to the edge of the cover concrete -- outer edge of cover concrete
    coverZ = BCol/2.0	# The distance from the section y-axis to the edge of the cover concrete -- outer edge of cover concrete
    
    coreY = coverY - coverCol
    coreZ = coverZ - coverCol
    coreY02 = coreY - 50 # Middle Rebar Distance
    coreZ02 = coreZ      # Middle Rebar Distance
    #MID_numBarsCol = numBarsCol/2
    
    nfCoreY = 16;			# number of fibers for concrete in y-direction -- core concrete
    nfCoreZ = 4;			# number of fibers for concrete in z-direction
    nfCoverY = 16;			# number of fibers for concrete in y-direction -- cover concrete
    nfCoverZ = 4;			# number of fibers for concrete in z-direction

    op.section('Fiber', ColSecTag)
    # Define the core patch
    op.patch('quad', IDconcCore, nfCoreZ, nfCoreY, -coreY,coreZ, -coreY,-coreZ, coreY,-coreZ, coreY, coreZ) # Define the concrete patch
    # Define the four cover patches
    op.patch('quad', IDconcCover, nfCoverZ, nfCoverY, -coverY,coverZ, -coreY, coreZ, coreY, coreZ, coverY,coverZ) # Define the concrete patch
    op.patch('quad', IDconcCover, nfCoverZ, nfCoverY, -coreY, -coreZ, -coverY, -coverZ, coverY, -coverZ, coreY, -coreZ) # Define the concrete patch
    op.patch('quad', IDconcCover, nfCoverZ, nfCoverY, -coverY, coverZ, -coverY, -coverZ, -coreY, -coreZ, -coreY, coreZ) # Define the concrete patch
    op.patch('quad', IDconcCover, nfCoverZ, nfCoverY, coreY, coreZ, coreY, -coreZ, coverY,-coverZ, coverY,coverZ) # Define the concrete patch
    # Define reinfocement layers    
    op.layer('straight', IDreinf, numBarsCol, barAreaCol, coreY, coreZ, coreY, -coreZ)# top layer reinforcement
    op.layer('straight', IDreinf, 2, barAreaCol, coreY02, coreZ02, coreY02, -coreZ02)# middle top layer reinforcement
    op.layer('straight', IDreinf, 2, barAreaCol, -coreY02, coreZ02, -coreY02, -coreZ02)# middle bottom layer reinforcement
    op.layer('straight', IDreinf, numBarsCol, barAreaCol, -coreY, coreZ, -coreY, -coreZ)# bottom layer reinfocement
    
    ColTransfTag = 1
    op.geomTransf('Linear', ColTransfTag)
    numIntgrPts = 5
    eleTag = 1
    op.element('nonlinearBeamColumn', eleTag, 1, 2, numIntgrPts, ColSecTag, ColTransfTag)

    #import InelasticFiberSection
    op.recorder('EnvelopeNode','-file', f"{SALAR_DIR}MD.txt" ,'-time','-node',2,'-dof',1,'disp');# max. displacements of free nodes
    op.recorder('EnvelopeNode','-file',f"{SALAR_DIR}MV.txt" ,'-time','-node',2,'-dof',1,'disp');# max. vel of free nodes
    op.recorder('EnvelopeNode','-file', f"{SALAR_DIR}MA.txt" ,'-time','-node',2,'-dof',1,'disp');# max. accel of free nodes	
    op.recorder('Node', '-file', f"{SALAR_DIR}DTH.txt",'-time', '-node', 2, '-dof', 1,2,3, 'disp')# Displacement Time History
    op.recorder('Node', '-file', f"{SALAR_DIR}VTH.txt",'-time', '-node', 2, '-dof', 1,2,3, 'vel')# Vel Time History
    op.recorder('Node', '-file', f"{SALAR_DIR}ATH.txt",'-time', '-node', 2, '-dof', 1,2,3, 'accel')# Accel Time History
    op.recorder('Node', '-file', f"{SALAR_DIR}BTH.txt",'-time', '-node', 1, '-dof', 1,2,3, 'reaction')# Base Shear Time History
    op.recorder('Element', '-file', f"{SALAR_DIR}FCol.txt",'-time', '-ele', 1, 'globalForce')
    op.recorder('Element', '-file', f"{SALAR_DIR}ForceColSec1.txt",'-time', '-ele', 1, 'section', 1, 'force')
    op.recorder('Element', '-file', f"{SALAR_DIR}DCol.txt','-time", '-ele', 1, 'deformations')

    #defining gravity loads
    op.timeSeries('Linear', 1)
    op.pattern('Plain', 1, 1)
    op.load(2, 0.0, -PCol, 0.0)

    Tol = 1e-8 # convergence tolerance for test
    NstepGravity = 10
    DGravity = 1/NstepGravity
    op.integrator('LoadControl', DGravity) # determine the next time step for an analysis
    op.numberer('Plain') # renumber dof's to minimize band-width (optimization), if you want to
    op.system('BandGeneral') # how to store and solve the system of equations in the analysis
    op.constraints('Plain') # how it handles boundary conditions
    op.test('NormDispIncr', Tol, 6) # determine if convergence has been achieved at the end of an iteration step
    op.algorithm('Newton') # use Newton's solution algorithm: updates tangent stiffness at every iteration
    op.analysis('Static') # define type of analysis static or transient
    op.analyze(NstepGravity) # apply gravity

    op.loadConst('-time', 0.0) #maintain constant gravity loads and reset time to zero

    #applying Dynamic Ground motion analysis
    GMdirection = 1
    GMfile = 'BM68elc.acc'
    GMfact = 1.0



    Lambda01 = op.eigen('-fullGenLapack', 1) # eigenvalue mode 1
    Lambda02 = op.eigen('-genBandArpack', 1) # eigenvalue mode 1
    #print(Lambda)
    Omega = math.pow(max(min(Lambda01), min(Lambda02)), 0.5)
    betaKcomm = 2 * (0.02/Omega)

    xDamp = 0.02				# 2% damping ratio
    alphaM = 0.0				# M-prop. damping; D = alphaM*M	
    betaKcurr = 0.0		# K-proportional damping;      +beatKcurr*KCurrent
    betaKinit = 0.0 # initial-stiffness proportional damping      +beatKinit*Kini

    op.rayleigh(alphaM,betaKcurr, betaKinit, betaKcomm) # RAYLEIGH damping

    # Uniform EXCITATION: acceleration input
    IDloadTag = 400			# load tag
    dt = 0.01			# time step for input ground motion
    GMfatt = 1.0			# data in input file is in g Unifts -- ACCELERATION TH
    maxNumIter = 10
    op.timeSeries('Path', 2, '-dt', dt, '-filePath', GMfile, '-factor', GMfact)
    op.pattern('UniformExcitation', IDloadTag, GMdirection, '-accel', 2) 

    op.wipeAnalysis()
    op.constraints('Transformation')
    op.numberer('Plain')
    op.system('BandGeneral')
    op.test('EnergyIncr', Tol, maxNumIter)
    op.algorithm('ModifiedNewton')

    NewmarkGamma = 0.5
    NewmarkBeta = 0.25
    op.integrator('Newmark', NewmarkGamma, NewmarkBeta)
    op.analysis('Transient')

    DtAnalysis = 0.01
    TmaxAnalysis = 10.0

    Nsteps =  int(TmaxAnalysis/ DtAnalysis)

    ok = op.analyze(Nsteps, DtAnalysis)

    tCurrent = op.getTime()

    # for gravity analysis, load control is fine, 0.1 is the load factor increment (http://opensees.berkeley.edu/wiki/index.php/Load_Control)

    test = {1:'NormDispIncr', 2: 'RelativeEnergyIncr', 4: 'RelativeNormUnbalance',5: 'RelativeNormDispIncr', 6: 'NormUnbalance'}
    algorithm = {1:'KrylovNewton', 2: 'SecantNewton' , 4: 'RaphsonNewton',5: 'PeriodicNewton', 6: 'BFGS', 7: 'Broyden', 8: 'NewtonLineSearch'}

    for i in test:
        for j in algorithm:

            if ok != 0:
                if j < 4:
                    op.algorithm(algorithm[j], '-initial')

                else:
                    op.algorithm(algorithm[j])

                op.test(test[i], Tol, 1000)
                ok = op.analyze(Nsteps, DtAnalysis)                            
                print(test[i], algorithm[j], ok)             
                if ok == 0:
                    break
            else:
                continue

    #u2 = op.nodeDisp(2, 1)
    #print("u2 = ", u2)
    print('Ground Motion Done.')
    op.wipe()
    


In [41]:
### --------------------------------------------------------------
###                COLUMN REBAR DIAMETER OPTIMIZATION
### --------------------------------------------------------------

# define section geometry
LCol = 30000.0 # [mm] column length
HCol = 300 # [mm] Column Depth
BCol = 300 # [mm] Column Width
coverCol = 50.0   # [mm] Column cover to reinforcing steel NA.
Weight = 1000000.0 # [N] superstructure weight
numBarsCol = 4  # number of longitudinal-reinforcement bars in column. (symmetric top & bot)
BD = 20  # [mm] Rebar Diamater
fc = -25.0 # [N/mm^2] Concrete Compressive Strength (+Tension, -Compression)
X = BD # Intial Guess

ESP = 1e-3 # Finite difference derivative Convergence Tolerance
TOLERANCE = 1e-6 # Convergence Tolerance
RESIDUAL = 100 # Convergence Residual 
IT = 0 # Intial Iteration
ITMAX = 100000 # Max. Iteration
TARGET_PGD = 20 # [mm] Target Demand Max. Abs. Displacement Time History

DATA_FILE ='C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\MD'  # MAX DISPLACEMENT
# monitor cpu time
import time
starttime = time.process_time()

### FIND THE OPTIMUM VALUE 
while (RESIDUAL > TOLERANCE):
    DYNAMIC_ANALYSIS(HCol, BCol, LCol, coverCol, Weight, numBarsCol, X, fc)
    time.sleep(10);# Sleep for 10 seconds
    F = MAXABS_FUN(DATA_FILE) - TARGET_PGD
    print('Current Max. Abs. displacement: ', MAXABS_FUN(DATA_FILE))
    # Evaluate at Xmain and Fmin
    Xmin = X - ESP
    DYNAMIC_ANALYSIS(HCol, BCol, LCol, coverCol, Weight, numBarsCol, Xmin, fc)
    time.sleep(10);# Sleep for 10 seconds
    Fmin = MAXABS_FUN(DATA_FILE) - TARGET_PGD
    # Evaluate at Xmax and Fmax
    Xmax = X + ESP
    DYNAMIC_ANALYSIS(HCol, BCol, LCol, coverCol, Weight, numBarsCol, Xmax, fc)
    time.sleep(10);# Sleep for 10 seconds
    Fmax = MAXABS_FUN(DATA_FILE) - TARGET_PGD
    
    DF = (Fmax-Fmin)/(2*ESP);# Calculate the Finite difference derivative of F
    DX = F / DF; # Calculate dx
    RESIDUAL = abs(DX); # Calculate residual
    print('RESIDUAL: ', RESIDUAL)
    X -= DX; # update X
    IT += 1; # update iteration
    if IT == ITMAX:
        print("\t\t Iteration reached to Max. Iteration")
        print("\t\t Change ESP and TOLERANCE for better Convergence")
        X = -1
        break;
    if RESIDUAL < TOLERANCE:
        print(f'\t\t Optimum Rebar Diamater: {X:.4f}')
        print(f'\t\t Iteration Counts: {IT}')
        print(f'\t\t Convergence Residual {RESIDUAL:.10e}')
    #print(X)
    
totaltime = time.process_time() - starttime
print(f'\nTotal time (s): {totaltime:.4f} \n\n')

Ground Motion Done.
Current Max. Abs. displacement:  17.5797
Ground Motion Done.
Ground Motion Done.
RESIDUAL:  1.6135333333332729
Ground Motion Done.
Current Max. Abs. displacement:  19.8789
Ground Motion Done.
Ground Motion Done.
RESIDUAL:  0.08970370370379357
Ground Motion Done.
Current Max. Abs. displacement:  19.9993
Ground Motion Done.
Ground Motion Done.
RESIDUAL:  0.0005185185185178362
Ground Motion Done.
Current Max. Abs. displacement:  20.0
Ground Motion Done.
Ground Motion Done.
RESIDUAL:  0.0
		 Optimum Rebar Diamater: 21.7038
		 Iteration Counts: 4
		 Convergence Residual 0.0000000000e+00

Total time (s): 1.7344 




In [ ]:
def HISROGRAM_BOXPLOT(X, HISTO_COLOR, LABEL):
    import numpy as np
    import matplotlib.pyplot as plt
    X = np.array(X)
    print("-------------------------")
    from scipy.stats import skew, kurtosis
    MINIMUM = np.min(X)
    MAXIMUM = np.max(X)
    #MODE = max(set(X), key=list(X).count)
    MEDIAN = np.quantile(X, .50)#q2
    MEAN = np.mean(X)
    STD = np.std(X)
    q1 = np.quantile(X, .25)
    q3 = np.quantile(X, .75)
    SKEW = skew(X)
    KURT = kurtosis(X)
    #SKEW = (MEAN - MODE) / STD
    #KURT = (np.mean((X - MEAN)**4) / STD**4)
    # Estimate confidence intervals of the output variable
    lower_bound = np.quantile(X, .05)
    upper_bound = np.quantile(X, .95)
    print("Box-Chart Datas: ")
    print(f'Minimum: {MINIMUM:.4f}')
    print(f'First quartile: {q1:.4f}')
    #print(f'Mode: {MODE:.4f}')
    print(f'Median: {MEDIAN:.4f}')
    print(f'Mean: {MEAN:.4f}')
    print(f'Std: {STD:.4f}')
    print(f'Third quartile: {q3:.4f}')
    print(f'Maximum: {MAXIMUM :.4f}')
    print(f'Skewness: {skew(X) :.4f}')
    print(f'kurtosis: {kurtosis(X) :.4f}')
    print(f"90% Confidence Interval: ({lower_bound:.4f}, {upper_bound:.4f})")
    print("-------------------------")

    plt.figure(figsize=(10,6))
    # Plot histogram of data
    count, bins, ignored = plt.hist(X, bins=100, color=HISTO_COLOR, density=True, align='mid')#, edgecolor="black"
    
    # Plot lognormal PDF
    x = np.linspace(min(bins), max(bins), 10000)
    pdf = (np.exp(-(x - MEAN)**2 / (2 * STD**2)) / (STD * np.sqrt(2 * np.pi)))
    plt.plot(x, pdf, linewidth=2, color='r', label="Normal PDF")
    
    # Plot vertical lines for risk measures
    plt.axvline(q1, color="black", linestyle="--", label=f"Quantile 0.25: {q1:.4f}")
    plt.axvline(MEDIAN, color="green", linestyle="--", label=f"Median: {MEDIAN:.4f}")
    plt.axvline(q3, color="black", linestyle="--", label=f"Quantile 0.75: {q3:.4f}")
    #plt.axvline(MODE, color="purple", linestyle="--", label=f"Mode: {MODE:.4f}")
    plt.axvline(MEAN, color="red", linestyle="--", label=f"Mean: {MEAN:.4f}")
    plt.axvline(MEAN-STD, color="blue", linestyle="--", label=f"Mean-Std: {MEAN-STD:.4f}")
    plt.axvline(MEAN+STD, color="blue", linestyle="--", label=f"Mean+Std: {MEAN+STD:.4f}")
    plt.xlabel(LABEL)
    plt.ylabel("Frequency")
    prob = np.sum(X > 0) / len(X)
    plt.title(f"Histogram - Probability of Positive {LABEL} is {100*prob:.2f} %")
    plt.legend()
    #plt.grid()
    plt.show()

    #Plot boxplot with outliers
    plt.figure(figsize=(10,6))
    plt.boxplot(X, vert=0)
    # Write the quartile data on the chart
    plt.text(q1, 1.05, f" Q1: {q1:.4f}")
    plt.text(MEDIAN, 1.1, f" Q2: {MEDIAN:.4f}")
    plt.text(q3, 1.05, f" Q3: {q3:.4f}")
    #plt.text(MODE, 1.15, f" Mode: {MODE:.4f}")
    
    #plt.text(MEAN, 0.9, f" Mean: {MEAN:.4f}")
    #plt.text(MEAN-STD, 0.9, f" Mean-Std: {MEAN-STD:.4f}")
    #plt.text(MEAN+STD, 0.9, f" Mean+Std: {MEAN+STD:.4f}")
    plt.scatter(MEAN, 1, color="red", marker="+", s=200, label=f"Mean: {MEAN:.4f}")
    plt.scatter(MEAN-STD, 1, color="green", marker="X", s=200, label=f"Mean-Std: {MEAN-STD:.4f}")
    plt.scatter(MEAN+STD, 1, color="blue", marker="*", s=200, label=f"Mean+Std:  {MEAN+STD:.4f}")
    plt.xlabel(LABEL)
    plt.ylabel("Data")
    plt.title(f"Boxplot of {LABEL}")
    plt.legend()
    plt.grid()
    plt.show()
    

###########################################

def PLOT_TIME_HIS(DTH, VTH, ATH, BTH):
    ## PLOT THE DATA
    import matplotlib.pyplot as plt
    # Create a 1x3 grid of subplots
    fig, axs = plt.subplots(4, 1, figsize=(14, 14))

    # Plot Displacement
    axs[0].plot(DTH, label='Displacement')
    axs[0].set_title(f'Last Analysis Displacement Time History - Max(Abs): {np.max(np.abs(DTH)):.6f}')
    axs[0].set_xlabel('Time')
    axs[0].set_ylabel('Displacement')
    axs[0].grid()

    # Plot Velocity
    axs[1].plot(VTH, label='Velocity', color='purple')
    axs[1].set_title(f'Last Analysis Velocity Time History - Max(Abs): {np.max(np.abs(ATH)):.6f}')
    axs[1].set_xlabel('Time')
    axs[1].set_ylabel('Velocity')
    axs[1].grid()

    # Plot Acceleration
    axs[2].plot(ATH, label='Acceleration', color='green')
    axs[2].set_title(f'Last Analysis Acceleration Time History - Max(Abs): {np.max(np.abs(ATH)):.6f}')
    axs[2].set_xlabel('Time')
    axs[2].set_ylabel('Acceleration')
    axs[2].grid()
    
    # Plot Base Shear
    axs[3].plot(BTH, label='Base Shear', color='orange')
    axs[3].set_title(f'Last Analysis Base Shear Time History - Max(Abs): {np.max(np.abs(BTH)):.6f}')
    axs[3].set_xlabel('Time')
    axs[3].set_ylabel('Base Shear')
    axs[3].grid()

    # Adjust layout
    plt.tight_layout()
    plt.show()

In [ ]:
### LOAD DATAS
import numpy as np
import pandas as pd

# Displacement Time History
filename = f'C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\DTH.txt'
data_collected = np.loadtxt(filename)
DTH = data_collected[:, 1]
# Velocity Time History
filename = 'C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\VTH.txt'
data_collected = np.loadtxt(filename)
VTH = data_collected[:, 1]
# Acceleration Time History
filename = 'C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\ATH.txt'
data_collected = np.loadtxt(filename)
ATH = data_collected[:, 1]
# Base Shear Time History
filename = 'C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\BTH.txt'
data_collected = np.loadtxt(filename)
BTH = data_collected[:, 1]

print(len(DTH), len(VTH),len(ATH))


In [ ]:
HISROGRAM_BOXPLOT(DTH, HISTO_COLOR='blue', LABEL='Displacement Time History')

In [ ]:
HISROGRAM_BOXPLOT(VTH, HISTO_COLOR='purple', LABEL='Velocity Time History')

In [ ]:
HISROGRAM_BOXPLOT(ATH, HISTO_COLOR='green', LABEL='Acceleration Time History')

In [ ]:
HISROGRAM_BOXPLOT(BTH, HISTO_COLOR='orange', LABEL='Base Shear Time History')

In [ ]:
# PLOT TIME HISTORY 
PLOT_TIME_HIS(DTH, VTH, ATH, BTH)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Plot shear force and bending moment
plt.figure(figsize=(12, 8))
plt.plot(DTH, BTH,color='black')
plt.xlabel('Displacement Time History')
plt.ylabel('Base Shear Time History')
plt.title('Base Shear and Displacement Time History')
plt.grid()
plt.show()


In [42]:
### --------------------------------------------------------------
###                    COLUMN DEPTH OPTIMIZATION
### --------------------------------------------------------------

# define section geometry
LCol = 30000.0 # [mm] column length
HCol = 300 # [mm] Column Depth
BCol = 300 # [mm] Column Width
coverCol = 50.0   # [mm] Column cover to reinforcing steel NA.
Weight = 1000000.0 # [N] superstructure weight
numBarsCol = 4  # number of longitudinal-reinforcement bars in column. (symmetric top & bot)
BD = 20  # [mm] Rebar Diamater
fc = -25.0 # [N/mm^2] Concrete Compressive Strength (+Tension, -Compression)
X = HCol # Intial Guess

ESP = 1e-3 # Finite difference derivative Convergence Tolerance
TOLERANCE = 1e-6 # Convergence Tolerance
RESIDUAL = 100 # Convergence Residual 
IT = 0 # Intial Iteration
ITMAX = 100000 # Max. Iteration
TARGET_PGD = 20 # [mm] Target Demand Max. Abs. Displacement Time History

DATA_FILE ='C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\MD'  # MAX DISPLACEMENT
# monitor cpu time
import time
starttime = time.process_time()

### FIND THE OPTIMUM VALUE 
while (RESIDUAL > TOLERANCE):
    DYNAMIC_ANALYSIS(X, BCol, LCol, coverCol, Weight, numBarsCol, BD, fc)
    time.sleep(10);# Sleep for 10 seconds
    F = MAXABS_FUN(DATA_FILE) - TARGET_PGD
    print('Current Max. Abs. displacement: ', MAXABS_FUN(DATA_FILE))
    # Evaluate at Xmain and Fmin
    Xmin = X - ESP
    DYNAMIC_ANALYSIS(Xmin, BCol, LCol, coverCol, Weight, numBarsCol, BD, fc)
    time.sleep(10);# Sleep for 10 seconds
    Fmin = MAXABS_FUN(DATA_FILE) - TARGET_PGD
    # Evaluate at Xmax and Fmax
    Xmax = X + ESP
    DYNAMIC_ANALYSIS(Xmax, BCol, LCol, coverCol, Weight, numBarsCol, BD, fc)
    time.sleep(10);# Sleep for 10 seconds
    Fmax = MAXABS_FUN(DATA_FILE) - TARGET_PGD
    # Calculate the Finite difference derivative of F
    DF = (Fmax-Fmin)/(2*ESP)
    # Calculate dx
    DX = F / DF
    # Calculate residual
    RESIDUAL = abs(DX)
    print('RESIDUAL: ', RESIDUAL)
    X -= DX # update X
    IT += 1 # update iteration
    if IT == ITMAX:
        print("\t\t Iteration reached to Max. Iteration")
        print("\t\t Change ESP and TOLERANCE for better Convergence")
        X = -1
        break;
    if RESIDUAL < TOLERANCE:
        print(f'\t\t Optimum Column Depth: {X:.4f}')
        print(f'\t\t Iteration Counts: {IT}')
        print(f'\t\t Convergence Residual {RESIDUAL:.10e}')
        
    #print(X)
    
totaltime = time.process_time() - starttime
print(f'\nTotal time (s): {totaltime:.4f} \n\n')

Ground Motion Done.
Current Max. Abs. displacement:  17.5797
Ground Motion Done.
Ground Motion Done.
RESIDUAL:  12.101500000028208
Ground Motion Done.
Current Max. Abs. displacement:  20.581
Ground Motion Done.
Ground Motion Done.
RESIDUAL:  2.324000000005414
Ground Motion Done.
Current Max. Abs. displacement:  19.9578
Ground Motion Done.
Ground Motion Done.
RESIDUAL:  0.1687999999991985
Ground Motion Done.
Current Max. Abs. displacement:  20.0023
Ground Motion Done.
Ground Motion Done.
RESIDUAL:  0.007666666666644955
Ground Motion Done.
Current Max. Abs. displacement:  20.0003
Ground Motion Done.
Ground Motion Done.
RESIDUAL:  0.0012
Ground Motion Done.
Current Max. Abs. displacement:  20.0
Ground Motion Done.
Ground Motion Done.
RESIDUAL:  0.0
		 Optimum Column Depth: 290.0626
		 Iteration Counts: 6
		 Convergence Residual 0.0000000000e+00

Total time (s): 2.7344 




In [ ]:
### LOAD DATAS
import numpy as np
import pandas as pd

# Displacement Time History
filename = f'C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\DTH.txt'
data_collected = np.loadtxt(filename)
DTH = data_collected[:, 1]
# Velocity Time History
filename = 'C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\VTH.txt'
data_collected = np.loadtxt(filename)
VTH = data_collected[:, 1]
# Acceleration Time History
filename = 'C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\ATH.txt'
data_collected = np.loadtxt(filename)
ATH = data_collected[:, 1]
# Base Shear Time History
filename = 'C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\BTH.txt'
data_collected = np.loadtxt(filename)
BTH = data_collected[:, 1]

print(len(DTH), len(VTH),len(ATH))


In [ ]:
HISROGRAM_BOXPLOT(DTH, HISTO_COLOR='blue', LABEL='Displacement Time History')

In [ ]:
HISROGRAM_BOXPLOT(VTH, HISTO_COLOR='purple', LABEL='Velocity Time History')

In [ ]:
HISROGRAM_BOXPLOT(ATH, HISTO_COLOR='green', LABEL='Acceleration Time History')

In [ ]:
HISROGRAM_BOXPLOT(BTH, HISTO_COLOR='orange', LABEL='Base Shear Time History')

In [ ]:
# PLOT TIME HISTORY 
PLOT_TIME_HIS(DTH, VTH, ATH, BTH)